In [ ]:
# for maths
import numpy as np
import pandas as pd
import scipy.stats as stats
import json

# for visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
from matplotlib.pyplot import figure

% matplotlib inline

In [ ]:
# loading the different datasets

streams1 = pd.read_json("/content/StreamingHistory0.json", orient='records')
streams2 = pd.read_json("/content/StreamingHistory1.json", orient='records')
streams3 = pd.read_json("/content/StreamingHistory2.json", orient='records')
stream_frames = [streams1, streams2, streams3]
streams = pd.concat(stream_frames)
streams.head()


,endTime,artistName,trackName,msPlayed
0,2021-07-18 22:11,Selena Gomez,Back To You,1582
1,2021-07-18 22:11,Lauv,Modern Loneliness,225776
2,2021-07-18 22:11,Adele,Send My Love (To Your New Lover),1951
3,2021-07-28 00:00,James Carter,Used To,159258
4,2021-07-28 00:03,James Carter,Used To,22527684


In [ ]:
# converting the df from long to wide
grouped_df = streams.groupby(["trackName", "artistName"])
grouped_lists = grouped_df["msPlayed"].apply(list)
grouped_lists = grouped_lists.reset_index()



# sum up total stream minutes per song and sort in descending order
grouped_lists["msPlayed"] = grouped_lists["msPlayed"].map(lambda msPlayed: sum(msPlayed)
)
grouped_lists.sort_values("msPlayed", ascending=False)


,trackName,artistName,msPlayed
6330,mirrorball,Taylor Swift,421024061
3668,Nico's Red Truck,Dijon,310066524
4998,Style,Taylor Swift,304099964
5637,Used To,James Carter,254511683
6232,emily,Jeremy Zucker,231341916
...,...,...,...
6090,You Belong With Me,Taylor Swift,0
2752,Juice Remix,Ycee,0
2349,Hymn for the Weekend - Seeb Remix,Coldplay,0
2347,Hush Hush,The Band CAMINO,0


Next steps:


*   create another df grouping the individual stream dates
*   Join the two dfs to create a wide version of the long df we originally had

*   Filter out any tracks that I listened to for less than 2 hours
*   visualize my top artists and bottom artists in a time vs artist graph, where the data points are individual songs (with labels)

*   do the same with my bottom 50 artists
*   create a streaming timeline per artist using dates (per month) ie

 *  see if I binge listen to an artist on a day vs if I shuffle through multiple in a session

 *   see if theres artists I listen to everyyday/ frequently and the habits ie do I listen to some artists on weekdays vs weekends







